<a href="https://colab.research.google.com/github/ychervonyi/reinforcement-learning-learning/blob/main/gridworld_example_chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bellman optimality equation for the value function is $v_*(s) = \mathrm{max}_a\sum_{s', r'} p(s', r|s, a)[r+\gamma v_*(s')]$. $s$ - any state, $s'$ - successor state, $r$ - rewards, $a$ - actions. $p(s', r|s, a)$ defines the dynamics of Markov decision processes $p(s', r|s, a) = \mathrm{Pr}(S_t=s', R_t=r| S_{t-1}=s,A_{t-1}=a)$.

These equations can be used to find the optimal policy. One way to solve them is using dynamic programming (see Chapters 3 and 4 in Sutton & Barto). In this notebook we will use some of the dynamic programming methods to solve grid world example (p. 60, example 3.5).

In the gridworld example there are 25 states (5x5 grid) $\mathcal{S} = \{0, 1..., 23, 24\}$. There are 4 actions for each cell - left, right, up, down $\mathcal{A}(s) = \{0, 1, 2, 3\}$ for all $s$. There are 4 rewards $\mathcal{R}\in \{-1, 0, 10, 5\}$.

Let's define all $p(s',r|s,a)$. 
For example, for state 0 (upper left corner) we get 
\begin{equation}
p(s',r|0,left) = \left\{ \begin{matrix}1 ~ \mathrm{if}~s'=0~\mathrm{and}~r=-1  \\ 0~\mathrm{otherwise} \end{matrix}\right.\\
p(s',r|0,up) = \left\{ \begin{matrix}1 ~ \mathrm{if}~s'=0~\mathrm{and}~r=-1  \\ 0~\mathrm{otherwise} \end{matrix}\right.\\
p(s',r|0,right) = \left\{ \begin{matrix}1 ~ \mathrm{if}~s'=1~\mathrm{and}~r=0  \\ 0~\mathrm{otherwise} \end{matrix}\right.\\
p(s',r|0,down) = \left\{ \begin{matrix}1 ~ \mathrm{if}~s'=5~\mathrm{and}~r=0  \\ 0~\mathrm{otherwise} \end{matrix}\right.\\
\end{equation}

In [2]:
import numpy as np

side = 5
states = np.array([[side * c + r for r in range(side)] for c in range(side)]) # coordinates [r, c]
rewards = {-1: 0, 0: 1, 10: 2, 5: 3} # [-1, 0, 10, 5]
actions = [[-1, 0], [1, 0], [0, -1], [0, 1]] # [dr, dc] = [up, down, left, right]

p = np.zeros((states.size, len(rewards), states.size, len(actions)))

# Assign p(s_p, r| s, a) for edges and all non-special cells
for r in range(side):
    for c in range(side):
        for i_a, (dr, dc) in enumerate(actions):
            # New state
            r_p, c_p = r + dr, c + dc
            # Edge - return to previous state and assing reward -1
            if r_p in (-1, side) or c_p in (-1, side):
                p[states[r][c]][rewards[-1]][states[r][c]][i_a] = 1
            # Non-edge (inside the grid) - for any action reward 0
            elif (r, c) not in ((0, 1), (0, 3)):
                p[states[r_p][c_p]][rewards[0]][states[r][c]][i_a] = 1
# A -> A' with reward +10
p[states[4][1]][rewards[10]][states[0][1]][:] = np.ones(len(actions))
# B -> B' with reward +5
p[states[2][3]][rewards[5]][states[0][3]][:] = np.ones(len(actions))

In [3]:
# Check 1: final state (0,0), reward -1, original state (0, 0). For left and up we should get probabilities 1
p[states[0][0]][rewards[-1]][states[0][0]]

array([1., 0., 1., 0.])

In [4]:
# Check 2: For this "up" action should have non zero probability
p[states[0][0]][rewards[0]][states[1][0]][:]

array([1., 0., 0., 0.])

In [34]:
# Print matrix
def print_v(A):
    for r in range(side):
        s = ""
        for c in range(side):
            s += f" {round(A[r*side+c], 1)}"
        print(s)

In [36]:
def iterative_policy_evaluation(gamma, epsilon):
    # Iterative policy evaluation (p. 75)
    # Value function
    V = np.random.uniform(low=-1, high=1, size=states.size)
    # V = np.zeros(states.size)

    steps, max_steps = 0, 20
    # Initial delta
    delta = epsilon
    # Keep max_steps in case it does not converge
    while delta >= epsilon and steps < max_steps:
        # Find max delta at each iteration
        delta = 0
        for s in range(states.size):
            v = V[s]
            v_new = 0
            for a_i in range(len(actions)):
                tmp = 0
                for s_p in range(states.size):
                    for r, r_i in rewards.items():
                        tmp += p[s_p][r_i][s][a_i] * (r + gamma * V[s_p])
                v_new = max(v_new, tmp)
            delta = max(delta, abs(v - v_new))
            V[s] = v_new
        print(f"delta: {delta}")
        # print_v(V)
        steps += 1
    print("Value function")
    print_v(V)

In [37]:
gamma = 0.5
threshold = 0.01
print(f"gamma: {gamma}, epsilon: {threshold}")
iterative_policy_evaluation(gamma, threshold)

print("=====================")
gamma = 0.9
print(f"gamma: {gamma}, epsilon: {threshold}")
iterative_policy_evaluation(gamma, threshold)

gamma: 0.5, epsilon: 0.01
delta: 11.322113353429108
delta: 5.204858144981433
delta: 2.188986666073701
delta: 1.1628991663516537
delta: 0.6177901821243132
delta: 0.32820103425354574
delta: 0.19702600992420294
delta: 0.1231412562026275
delta: 0.07696328512664152
delta: 0.04810205320415051
delta: 0.030063783252593623
delta: 0.018789864532871903
delta: 0.011743665333044717
delta: 0.007339790833153614
Value function
 9.6 19.2 9.6 10.7 5.3
 4.8 9.6 4.8 5.3 2.7
 2.4 4.8 2.4 2.7 1.3
 1.2 2.4 1.2 1.3 0.7
 0.6 1.2 0.6 0.7 0.3
gamma: 0.9, epsilon: 0.01
delta: 10.594005395254346
delta: 14.299093878510076
delta: 21.312656434980482
delta: 31.76630128977407
delta: 47.34735440939534
delta: 70.57075827365966
delta: 114.0838556922314
delta: 185.84260092264498
delta: 302.73759690298857
delta: 493.1595453549685
delta: 803.3568993832437
delta: 1308.6683890953045
delta: 2131.8208058362507
delta: 3472.7360927072514
delta: 5657.087095020113
delta: 9215.394877787767
delta: 15011.878255916276
delta: 24454.34967

My experiments show that this example does not converge for $\gamma > 0.7$. Probably because we can have an infinite loop - a wormhole is feeding itself - going from A to A' and back infinitevily.